In [1]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=25
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:200]
for i in range(9):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    all=np.concatenate((x[:200],all))

mnist_images=all
np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,256,448,496,510])
min_array=np.array([0.05,0.01,0.005])
layer_array=np.array([5,10,20,50]) 
print(NUM_QUBITS)
print(T)
zero = torch.zeros(BATCH_SIZE, 2**NUM_QUBITS-ld_dim).to(device)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    

                    # concatenate the two tensors along the second dimension
                    input_batch = torch.cat((input_batch, zero), dim=1)
                    target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer} QC{qc}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'ancilla_{Q_ANCILLA}/all_thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'ancilla_{Q_ANCILLA}/all_loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

9
25
(26,)
0
T=25 Epoch: 1/40 - Loss: 0.8979 b_loss=0.8979 - T: 31.89s/epoch ,tempo_previto=1275.03 min0.05 nl5 QC0
1
T=25 Epoch: 2/40 - Loss: 0.8277 b_loss=0.8277 - T: 34.05s/epoch ,tempo_previto=1360.81 min0.05 nl5 QC0
2
T=25 Epoch: 3/40 - Loss: 0.7418 b_loss=0.7418 - T: 34.02s/epoch ,tempo_previto=1359.03 min0.05 nl5 QC0
3
T=25 Epoch: 4/40 - Loss: 0.7035 b_loss=0.7035 - T: 33.56s/epoch ,tempo_previto=1340.32 min0.05 nl5 QC0
4
T=25 Epoch: 5/40 - Loss: 0.6705 b_loss=0.6705 - T: 32.79s/epoch ,tempo_previto=1308.87 min0.05 nl5 QC0
5
T=25 Epoch: 6/40 - Loss: 0.6516 b_loss=0.6516 - T: 32.83s/epoch ,tempo_previto=1309.90 min0.05 nl5 QC0
6
T=25 Epoch: 7/40 - Loss: 0.6610 b_loss=0.6516 - T: 32.75s/epoch ,tempo_previto=1306.13 min0.05 nl5 QC0
7
T=25 Epoch: 8/40 - Loss: 0.6486 b_loss=0.6486 - T: 32.92s/epoch ,tempo_previto=1312.51 min0.05 nl5 QC0
8
T=25 Epoch: 9/40 - Loss: 0.6550 b_loss=0.6486 - T: 32.64s/epoch ,tempo_previto=1300.83 min0.05 nl5 QC0
9
T=25 Epoch: 10/40 - Loss: 0.6314 b_loss=0.

T=25 Epoch: 38/40 - Loss: 0.3306 b_loss=0.3104 - T: 33.08s/epoch ,tempo_previto=1191.99 min0.01 nl5 QC0
38
T=25 Epoch: 39/40 - Loss: 0.3507 b_loss=0.3104 - T: 32.94s/epoch ,tempo_previto=1186.24 min0.01 nl5 QC0
39
T=25 Epoch: 40/40 - Loss: 0.3365 b_loss=0.3104 - T: 32.99s/epoch ,tempo_previto=1187.54 min0.01 nl5 QC0
(26,)
0
T=25 Epoch: 1/40 - Loss: 0.8325 b_loss=0.8325 - T: 33.70s/epoch ,tempo_previto=1122.83 min0.005 nl5 QC0
1
T=25 Epoch: 2/40 - Loss: 0.6246 b_loss=0.6246 - T: 32.92s/epoch ,tempo_previto=1096.34 min0.005 nl5 QC0
2
T=25 Epoch: 3/40 - Loss: 0.4665 b_loss=0.4665 - T: 33.07s/epoch ,tempo_previto=1100.67 min0.005 nl5 QC0
3
T=25 Epoch: 4/40 - Loss: 0.3880 b_loss=0.3880 - T: 32.95s/epoch ,tempo_previto=1096.08 min0.005 nl5 QC0
4
T=25 Epoch: 5/40 - Loss: 0.3321 b_loss=0.3321 - T: 32.80s/epoch ,tempo_previto=1090.66 min0.005 nl5 QC0
5
T=25 Epoch: 6/40 - Loss: 0.3088 b_loss=0.3088 - T: 33.27s/epoch ,tempo_previto=1105.52 min0.005 nl5 QC0
6
T=25 Epoch: 7/40 - Loss: 0.3020 b_loss

T=25 Epoch: 34/40 - Loss: 0.6074 b_loss=0.6004 - T: 33.00s/epoch ,tempo_previto=1279.33 min0.05 nl5 QC256
34
T=25 Epoch: 35/40 - Loss: 0.6199 b_loss=0.6004 - T: 33.27s/epoch ,tempo_previto=1289.34 min0.05 nl5 QC256
35
T=25 Epoch: 36/40 - Loss: 0.6320 b_loss=0.6004 - T: 32.90s/epoch ,tempo_previto=1274.22 min0.05 nl5 QC256
36
T=25 Epoch: 37/40 - Loss: 0.6364 b_loss=0.6004 - T: 33.13s/epoch ,tempo_previto=1282.60 min0.05 nl5 QC256
37
T=25 Epoch: 38/40 - Loss: 0.6098 b_loss=0.6004 - T: 33.44s/epoch ,tempo_previto=1293.96 min0.05 nl5 QC256
38
T=25 Epoch: 39/40 - Loss: 0.6168 b_loss=0.6004 - T: 33.29s/epoch ,tempo_previto=1287.88 min0.05 nl5 QC256
39
T=25 Epoch: 40/40 - Loss: 0.6357 b_loss=0.6004 - T: 33.08s/epoch ,tempo_previto=1279.06 min0.05 nl5 QC256
(26,)
0
T=25 Epoch: 1/40 - Loss: 0.8893 b_loss=0.8893 - T: 33.02s/epoch ,tempo_previto=1188.20 min0.01 nl5 QC256
1
T=25 Epoch: 2/40 - Loss: 0.7108 b_loss=0.7108 - T: 32.88s/epoch ,tempo_previto=1182.47 min0.01 nl5 QC256
2
T=25 Epoch: 3/40 -

T=25 Epoch: 30/40 - Loss: 0.2786 b_loss=0.2654 - T: 33.22s/epoch ,tempo_previto=1068.53 min0.005 nl5 QC256
30
T=25 Epoch: 31/40 - Loss: 0.2887 b_loss=0.2654 - T: 33.01s/epoch ,tempo_previto=1061.40 min0.005 nl5 QC256
31
T=25 Epoch: 32/40 - Loss: 0.2935 b_loss=0.2654 - T: 33.06s/epoch ,tempo_previto=1062.21 min0.005 nl5 QC256
32
T=25 Epoch: 33/40 - Loss: 0.2682 b_loss=0.2654 - T: 32.96s/epoch ,tempo_previto=1058.49 min0.005 nl5 QC256
33
T=25 Epoch: 34/40 - Loss: 0.2780 b_loss=0.2654 - T: 34.44s/epoch ,tempo_previto=1105.48 min0.005 nl5 QC256
34
T=25 Epoch: 35/40 - Loss: 0.2778 b_loss=0.2654 - T: 33.17s/epoch ,tempo_previto=1064.33 min0.005 nl5 QC256
35
T=25 Epoch: 36/40 - Loss: 0.2766 b_loss=0.2654 - T: 33.07s/epoch ,tempo_previto=1060.36 min0.005 nl5 QC256
36
T=25 Epoch: 37/40 - Loss: 0.2824 b_loss=0.2654 - T: 33.20s/epoch ,tempo_previto=1063.94 min0.005 nl5 QC256
37
T=25 Epoch: 38/40 - Loss: 0.2855 b_loss=0.2654 - T: 33.18s/epoch ,tempo_previto=1062.90 min0.005 nl5 QC256
38
T=25 Epoch

T=25 Epoch: 26/40 - Loss: 0.3461 b_loss=0.3170 - T: 33.08s/epoch ,tempo_previto=1154.59 min0.01 nl5 QC448
26
T=25 Epoch: 27/40 - Loss: 0.3296 b_loss=0.3170 - T: 33.26s/epoch ,tempo_previto=1160.37 min0.01 nl5 QC448
27
T=25 Epoch: 28/40 - Loss: 0.3441 b_loss=0.3170 - T: 33.15s/epoch ,tempo_previto=1155.77 min0.01 nl5 QC448
28
T=25 Epoch: 29/40 - Loss: 0.3412 b_loss=0.3170 - T: 33.20s/epoch ,tempo_previto=1157.19 min0.01 nl5 QC448
29
T=25 Epoch: 30/40 - Loss: 0.3357 b_loss=0.3170 - T: 33.03s/epoch ,tempo_previto=1150.62 min0.01 nl5 QC448
30
T=25 Epoch: 31/40 - Loss: 0.3180 b_loss=0.3170 - T: 33.04s/epoch ,tempo_previto=1150.49 min0.01 nl5 QC448
31
T=25 Epoch: 32/40 - Loss: 0.3323 b_loss=0.3170 - T: 32.88s/epoch ,tempo_previto=1144.39 min0.01 nl5 QC448
32
T=25 Epoch: 33/40 - Loss: 0.3524 b_loss=0.3170 - T: 32.93s/epoch ,tempo_previto=1145.56 min0.01 nl5 QC448
33
T=25 Epoch: 34/40 - Loss: 0.3265 b_loss=0.3170 - T: 34.12s/epoch ,tempo_previto=1186.29 min0.01 nl5 QC448
34
T=25 Epoch: 35/40 -

...

T=25 Epoch: 32/40 - Loss: 0.0279 b_loss=0.0279 - T: 332.22s/epoch ,tempo_previto=708.74 min0.005 nl50 QC256
32
T=25 Epoch: 33/40 - Loss: 0.0307 b_loss=0.0279 - T: 329.27s/epoch ,tempo_previto=696.96 min0.005 nl50 QC256
33
T=25 Epoch: 34/40 - Loss: 0.0266 b_loss=0.0266 - T: 331.57s/epoch ,tempo_previto=696.30 min0.005 nl50 QC256
34
T=25 Epoch: 35/40 - Loss: 0.0294 b_loss=0.0266 - T: 330.29s/epoch ,tempo_previto=688.11 min0.005 nl50 QC256
35
T=25 Epoch: 36/40 - Loss: 0.0307 b_loss=0.0266 - T: 329.49s/epoch ,tempo_previto=680.94 min0.005 nl50 QC256
36
T=25 Epoch: 37/40 - Loss: 0.0308 b_loss=0.0266 - T: 329.21s/epoch ,tempo_previto=674.87 min0.005 nl50 QC256
37
T=25 Epoch: 38/40 - Loss: 0.0302 b_loss=0.0266 - T: 329.96s/epoch ,tempo_previto=670.93 min0.005 nl50 QC256
38
T=25 Epoch: 39/40 - Loss: 0.0303 b_loss=0.0266 - T: 328.56s/epoch ,tempo_previto=662.59 min0.005 nl50 QC256
39
T=25 Epoch: 40/40 - Loss: 0.0316 b_loss=0.0266 - T: 331.16s/epoch ,tempo_previto=662.33 min0.005 nl50 QC256
(26,

T=25 Epoch: 27/40 - Loss: 0.0424 b_loss=0.0420 - T: 142.66s/epoch ,tempo_previto=696.64 min0.01 nl50 QC448
27
T=25 Epoch: 28/40 - Loss: 0.0443 b_loss=0.0420 - T: 142.91s/epoch ,tempo_previto=695.50 min0.01 nl50 QC448
28
T=25 Epoch: 29/40 - Loss: 0.0401 b_loss=0.0401 - T: 143.14s/epoch ,tempo_previto=694.24 min0.01 nl50 QC448
29
T=25 Epoch: 30/40 - Loss: 0.0381 b_loss=0.0381 - T: 145.06s/epoch ,tempo_previto=701.14 min0.01 nl50 QC448
30
T=25 Epoch: 31/40 - Loss: 0.0419 b_loss=0.0381 - T: 143.23s/epoch ,tempo_previto=689.89 min0.01 nl50 QC448
31
T=25 Epoch: 32/40 - Loss: 0.0368 b_loss=0.0368 - T: 142.41s/epoch ,tempo_previto=683.54 min0.01 nl50 QC448
32
T=25 Epoch: 33/40 - Loss: 0.0396 b_loss=0.0368 - T: 144.24s/epoch ,tempo_previto=689.97 min0.01 nl50 QC448
33
T=25 Epoch: 34/40 - Loss: 0.0372 b_loss=0.0368 - T: 142.95s/epoch ,tempo_previto=681.37 min0.01 nl50 QC448
34
T=25 Epoch: 35/40 - Loss: 0.0402 b_loss=0.0368 - T: 144.20s/epoch ,tempo_previto=684.96 min0.01 nl50 QC448
35
T=25 Epoch

T=25 Epoch: 22/40 - Loss: 0.2719 b_loss=0.2593 - T: 142.16s/epoch ,tempo_previto=1085.17 min0.05 nl50 QC496
22
T=25 Epoch: 23/40 - Loss: 0.2725 b_loss=0.2593 - T: 142.96s/epoch ,tempo_previto=1088.91 min0.05 nl50 QC496
23
T=25 Epoch: 24/40 - Loss: 0.2968 b_loss=0.2593 - T: 143.64s/epoch ,tempo_previto=1091.68 min0.05 nl50 QC496
24
T=25 Epoch: 25/40 - Loss: 0.2830 b_loss=0.2593 - T: 143.06s/epoch ,tempo_previto=1084.87 min0.05 nl50 QC496
25
T=25 Epoch: 26/40 - Loss: 0.2618 b_loss=0.2593 - T: 142.41s/epoch ,tempo_previto=1077.55 min0.05 nl50 QC496
26
T=25 Epoch: 27/40 - Loss: 0.2728 b_loss=0.2593 - T: 143.22s/epoch ,tempo_previto=1081.32 min0.05 nl50 QC496
27
T=25 Epoch: 28/40 - Loss: 0.2701 b_loss=0.2593 - T: 142.42s/epoch ,tempo_previto=1072.87 min0.05 nl50 QC496
28
T=25 Epoch: 29/40 - Loss: 0.2851 b_loss=0.2593 - T: 143.15s/epoch ,tempo_previto=1076.03 min0.05 nl50 QC496
29
T=25 Epoch: 30/40 - Loss: 0.2768 b_loss=0.2593 - T: 142.45s/epoch ,tempo_previto=1068.37 min0.05 nl50 QC496
30
T

T=25 Epoch: 17/40 - Loss: 0.0782 b_loss=0.0719 - T: 152.89s/epoch ,tempo_previto=160.54 min0.005 nl50 QC496
17
T=25 Epoch: 18/40 - Loss: 0.0727 b_loss=0.0719 - T: 149.57s/epoch ,tempo_previto=154.56 min0.005 nl50 QC496
18
T=25 Epoch: 19/40 - Loss: 0.0747 b_loss=0.0719 - T: 146.38s/epoch ,tempo_previto=148.82 min0.005 nl50 QC496
19
T=25 Epoch: 20/40 - Loss: 0.0721 b_loss=0.0719 - T: 143.83s/epoch ,tempo_previto=143.83 min0.005 nl50 QC496
20
T=25 Epoch: 21/40 - Loss: 0.0711 b_loss=0.0711 - T: 142.53s/epoch ,tempo_previto=140.15 min0.005 nl50 QC496
21
T=25 Epoch: 22/40 - Loss: 0.0702 b_loss=0.0702 - T: 142.86s/epoch ,tempo_previto=138.10 min0.005 nl50 QC496
22
T=25 Epoch: 23/40 - Loss: 0.0707 b_loss=0.0702 - T: 142.31s/epoch ,tempo_previto=135.19 min0.005 nl50 QC496
23
T=25 Epoch: 24/40 - Loss: 0.0694 b_loss=0.0694 - T: 143.04s/epoch ,tempo_previto=133.51 min0.005 nl50 QC496
24
T=25 Epoch: 25/40 - Loss: 0.0677 b_loss=0.0677 - T: 143.44s/epoch ,tempo_previto=131.49 min0.005 nl50 QC496
25
T

T=25 Epoch: 12/40 - Loss: 0.2614 b_loss=0.2604 - T: 143.00s/epoch ,tempo_previto=543.40 min0.01 nl50 QC510
12
T=25 Epoch: 13/40 - Loss: 0.2615 b_loss=0.2604 - T: 143.66s/epoch ,tempo_previto=543.50 min0.01 nl50 QC510
13
T=25 Epoch: 14/40 - Loss: 0.2458 b_loss=0.2458 - T: 141.97s/epoch ,tempo_previto=534.74 min0.01 nl50 QC510
14
T=25 Epoch: 15/40 - Loss: 0.2470 b_loss=0.2458 - T: 142.30s/epoch ,tempo_previto=533.61 min0.01 nl50 QC510
15
T=25 Epoch: 16/40 - Loss: 0.2427 b_loss=0.2427 - T: 143.24s/epoch ,tempo_previto=534.75 min0.01 nl50 QC510
16
T=25 Epoch: 17/40 - Loss: 0.2423 b_loss=0.2423 - T: 143.63s/epoch ,tempo_previto=533.84 min0.01 nl50 QC510
17
T=25 Epoch: 18/40 - Loss: 0.2525 b_loss=0.2423 - T: 142.90s/epoch ,tempo_previto=528.73 min0.01 nl50 QC510
18
T=25 Epoch: 19/40 - Loss: 0.2703 b_loss=0.2423 - T: 143.65s/epoch ,tempo_previto=529.12 min0.01 nl50 QC510
19
T=25 Epoch: 20/40 - Loss: 0.2509 b_loss=0.2423 - T: 144.06s/epoch ,tempo_previto=528.22 min0.01 nl50 QC510
20
T=25 Epoch